In [155]:
import pandas as pd
import numpy as np
import os
import openpyxl as xl

import re

import warnings
warnings.filterwarnings('ignore')



from Utilities import FileIO

# Load File

In [137]:
stagingPath = "./staging/"
allFileNames = os.listdir(stagingPath)

allYears = [int(re.findall(r'\d+', fileName)[0]) for fileName in allFileNames]
# allYears


In [138]:



def loadFile(year):
    strYear = str(year)
    matchingFile = [f for f in allFileNames if strYear in f][0]
    matchingPath = os.path.join(stagingPath, matchingFile)
    loadedWorkbook= xl.load_workbook(matchingPath, data_only=True)
    return loadedWorkbook

wb = loadFile(2021)
wb.sheetnames

['CoverSheet', 'Cost1', 'Cost2', 'Cost3', 'Volume1', 'Volume2']

In [139]:
def stripAllCells(frame):
    def stripCell(row):
        return str(row).strip()
    
    for col in frame.columns:
        frame[col] = frame[col].apply(lambda row: stripCell(row))
    return frame



def dataFrameFromTab(workbook, sheetname):
    frame = pd.DataFrame(workbook[sheetname].values)

    frame = stripAllCells(frame)
    return frame 

# df = dataFrameFromTab(wb, 'Cost2')
df = dataFrameFromTab(wb, 'Cost3')

# df = dataFrameFromTab(wb, 'Cost1')

df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,None,None,None,None,None,None,None,None,None,None,None,None
1,None,PUBLIC COST AND REVENUE ANALYSIS\n Fiscal Year...,None,None,None,None,None,None,None,None,None,None
2,None,None,(in millions),None,None,(per piece),None,None,None,None,None,None
3,None,None,None,Attributable,Volume,Revenue,Attributable,Volume,Contribution,Cost,None,None
4,None,Mail Classes and Products,Revenue,Cost,Variable Cost,$,Cost $,Variable,$,Coverage,None,None


In [140]:

columnIdDict_cost={
    "product":["Flats", "COD", "Total Express Mail", "Total Priority Mail Express"], 
    "revenue" :["Revenue", "A"], 
    "attributable_cost" :["Attributable", "Cost"], 
    "volume_variable_cost" :["Volume", "Variable Cost"], 
    "product_specific_cost" :["Product", "Specific Cost"], 
    "rev_pc" :["Revenue", "$"], 
    "attributable_cost_pc" :["Attributable", "Cost $"], 
    "cont_pc" :["(D-E)"], 
    "cost_coverage" :["Cost", "Coverage"], 
}

def identifyColumns_cost(frame):
    frame.columns = [str(c) for c in frame.columns]
    
    for keyDict, valDict in columnIdDict_cost.items():
        for col in frame.columns:
            colList = list(frame[col])
            if keyDict =="product":
                if any(element in colList for element in valDict):
                    frame.rename(columns={col: keyDict}, inplace=True)
                    continue
            if keyDict  =="attributable_cost":
                atlternateMatchString_ac = ["Vol Var &", "Prod Spec"]
                if all(element in colList for element in atlternateMatchString_ac):
                    frame.rename(columns={col: keyDict}, inplace=True)
            if keyDict  =="cost_pc":
                atlternateMatchString_ac_pc = ["Vol Var &", "Prod Spec $"]
                if all(element in colList for element in atlternateMatchString_ac_pc):
                    frame.rename(columns={col: keyDict}, inplace=True)
            if all(element in colList for element in valDict):
                frame.rename(columns={col: keyDict}, inplace=True)

            
    return frame


colsMatched = identifyColumns_cost(df)
colsMatched.head()


,0,product,revenue,attributable_cost,volume_variable_cost,rev_pc,attributable_cost_pc,7,cont_pc,cost_coverage,10,11
0,None,None,None,None,None,None,None,None,None,None,None,None
1,None,PUBLIC COST AND REVENUE ANALYSIS\n Fiscal Year...,None,None,None,None,None,None,None,None,None,None
2,None,None,(in millions),None,None,(per piece),None,None,None,None,None,None
3,None,None,None,Attributable,Volume,Revenue,Attributable,Volume,Contribution,Cost,None,None
4,None,Mail Classes and Products,Revenue,Cost,Variable Cost,$,Cost $,Variable,$,Coverage,None,None


In [141]:
columnIdDict_volume={
    "product":["Flats", "COD", "Total Express Mail", "Total Priority Mail Express"], 
    "volume" :["Pieces", "Transactions"], 
    "weight" :["Weight in"], 
    "weight_pc" :["Weight per"], 
}


def identifyColumns_volume(frame):
    frame.columns = [str(c) for c in frame.columns]
    
    for keyDict, valDict in columnIdDict_volume.items():
        for col in frame.columns:
            colList = list(frame[col])
            if keyDict in ["product", "volume"]:
                if any(element in colList for element in valDict):
                    frame.rename(columns={col: keyDict}, inplace=True)
                    continue
            if all(element in colList for element in valDict):
                frame.rename(columns={col: keyDict}, inplace=True)
            
    return frame



In [142]:


def testColumnMatch(colTestList, worksheetTabs):
        for year in allYears: 
            currentFile = loadFile(year)

            if "revenue" in colTestList:
                 columnFunc = identifyColumns_cost
                 columnDict = columnIdDict_cost
            else: 
                 columnFunc = identifyColumns_volume
                 columnDict = columnIdDict_volume

            for tab in worksheetTabs: 
                if tab in currentFile.sheetnames:
                    currentFrame = dataFrameFromTab(currentFile, tab)
                    currentFrame_cols = columnFunc(currentFrame)
                    # return currentFrame_cols
                    missingCols = [el for el in colTestList if not el in currentFrame_cols.columns]
                    if  len(missingCols)>0:
                        print(year, tab, missingCols) 


testColumnMatch(["product",    "revenue" ,     "attributable_cost" ], 
                ['Cost1', 'Cost2', 'Cost3'] )

    

In [143]:

# wb = loadFile(2008)
# df = dataFrameFromTab(wb, 'Volume1')
# df_strip = stripAllCells(df)
# df_volCols = identifyColumns_volume(df_strip,columnIdDict_volume )
# df_volCols.head()


testColumnMatch(list(columnIdDict_volume.keys()), 
                ['Volume1','Volume2', 'Volume3'] )

# Union All Tabs _Cost


In [144]:
# testFile = loadFile(2008)
# testFile = loadFile(2022)
# testFile = loadFile(2021)
# testFile = loadFile(2016)
# testFile = loadFile(2020)






def unionTabs_cost(fileInput):
    costTabs = [c for c in fileInput.sheetnames if "Cost" in c]
    rez = pd.DataFrame()
    # for tab in costTabs[1]:
    for tab in costTabs:
        # print("attempting tab", tab)
        frame_raw = dataFrameFromTab(fileInput, tab)
        frame_columnsMatched = identifyColumns_cost(frame_raw)
        allCostColumns = columnIdDict_cost.keys()
        missingCols = [c for c in  allCostColumns if c not in frame_columnsMatched]
        for missingCol in missingCols:
            frame_columnsMatched[missingCol] = "None"

        frame_importantCols = frame_columnsMatched[allCostColumns]

        frame_dropIdx = frame_importantCols.reset_index(drop=True) 
        rez = rez.reset_index(drop=True)
        rez = pd.concat([rez, frame_dropIdx])
        # print("finished", tab)
    
    return rez[allCostColumns]


testFile21 = loadFile(2021)
df_costTest21 = unionTabs_cost(testFile21)
df_costTest21
# print(testFile20.columns, testFile21.columns)


,product,revenue,attributable_cost,volume_variable_cost,product_specific_cost,rev_pc,attributable_cost_pc,cont_pc,cost_coverage
0,None,None,None,None,None,None,None,None,None
1,PUBLIC COST AND REVENUE ANALYSIS\n Fiscal Year...,None,None,None,None,None,None,None,None
2,None,(in millions),None,None,None,(per piece),None,None,None
3,None,None,Attributable,Volume,None,Revenue,Attributable,Contribution,Cost
4,Mail Classes and Products,Revenue,Cost,Variable Cost,None,$,Cost $,$,Coverage
...,...,...,...,...,...,...,...,...,...
153,None,None,None,None,None,None,None,None,None
154,None,None,None,None,None,None,None,None,None
155,None,None,None,None,None,None,None,None,None
156,None,None,None,None,None,None,None,None,None


In [145]:
def unionTabs_costAGAIN(fileInput):
    costTabs = [c for c in fileInput.sheetnames if "Cost" in c]
    rez = pd.DataFrame()

    tab = costTabs[1]

    frame_raw = dataFrameFromTab(fileInput, tab)
    frame_columnsMatched = identifyColumns_cost(frame_raw)
    # return frame_columnsMatched
    allCostColumns = columnIdDict_cost.keys()
    missingCols = [c for c in  allCostColumns if c not in frame_columnsMatched]
    for missingCol in missingCols:
        frame_columnsMatched[missingCol] = "None"

    frame_importantCols = frame_columnsMatched[allCostColumns]
    # return frame_importantCols

    frame_dropIdx = frame_importantCols.reset_index(drop=True) 
    # return frame_dropIdx
    rez = rez.reset_index(drop=True)
    rez = pd.concat([rez, frame_dropIdx])
    
    return rez[allCostColumns]

testFile21 = loadFile(2021)
df_costTestAGAIN = unionTabs_costAGAIN(testFile21)
df_costTestAGAIN

,product,revenue,attributable_cost,volume_variable_cost,product_specific_cost,rev_pc,attributable_cost_pc,cont_pc,cost_coverage
0,None,None,None,None,None,None,None,None,None
1,PUBLIC COST AND REVENUE ANALYSIS\n Fiscal Year...,None,None,None,None,None,None,None,None
2,None,(in millions),None,None,None,(per piece),None,None,None
3,None,None,Attributable,Volume,None,Revenue,Attributable,Contribution,Cost
4,Mail Classes and Products,Revenue,Cost,Variable Cost,None,$,Cost $,$,Coverage
...,...,...,...,...,...,...,...,...,...
153,None,None,None,None,None,None,None,None,None
154,None,None,None,None,None,None,None,None,None
155,None,None,None,None,None,None,None,None,None
156,None,None,None,None,None,None,None,None,None


# Union All Tabs _Volume

In [146]:
# testFile = loadFile(2008)
# testFile = loadFile(2009)
# testFile = loadFile(2010)
# testFile = loadFile(2022)
testFile = loadFile(2021)


def unionTabs_volume(fileInput):
    volTabs = [c for c in fileInput.sheetnames if "Volume" in c]
    rez = pd.DataFrame()
    for tab in volTabs:
        frame_raw = dataFrameFromTab(fileInput, tab)
        frame_columnsMatched = identifyColumns_volume(frame_raw)
        allVolumeColumns = columnIdDict_volume.keys()
        missingCols = [c for c in  allVolumeColumns if c not in frame_columnsMatched]
        for missingCol in missingCols:
            frame_columnsMatched[missingCol] = "None"
        frame_importantCols = frame_columnsMatched[allVolumeColumns]

        rez = pd.concat([rez, frame_importantCols])
        rez = rez.dropna(subset = allVolumeColumns, how="all")
    
    return rez[allVolumeColumns]

df_volTest = unionTabs_volume(testFile)
#df_volTest

# Drop Blank Rows

In [147]:
def dropBlankRows(frame):
    measureCols = [c for c in frame.columns if c !='fy']
    frame = frame[~(frame[measureCols]=="None").all(axis=1)]
    return frame


# Drop Blank Product Name

In [148]:
def dropBlankProduct(frame):
    frame = frame[frame['product']!="None"]

    return frame



#  Aggregate Extract Funcs

In [153]:
df_volume = pd.DataFrame()
df_cost = pd.DataFrame()


for year in allYears:
    # print("attempting year", year)
    currentFile = loadFile(year)
    
    currentFrame_cost = unionTabs_cost(currentFile)
    currentFrame_volume = unionTabs_volume(currentFile)

    currentFrame_cost['fy'] = year
    currentFrame_volume['fy'] = year

    df_cost = pd.concat([df_cost, currentFrame_cost], axis=0, ignore_index=True)
    df_volume = pd.concat([df_volume, currentFrame_volume], axis=0, ignore_index=True)

    df_cost = dropBlankRows(df_cost)
    df_volume = dropBlankRows(df_volume)

    df_cost = dropBlankProduct(df_cost)
    df_volume = dropBlankProduct(df_volume)

    df_cost = df_cost.drop_duplicates(keep="first")
    df_volume = df_volume.drop_duplicates(keep="first")





# Upload Files

In [156]:


FileIO.exportFile(df_cost, "extractFinished_cost.csv")
FileIO.exportFile(df_volume, "extractFinished_volume.csv")

# Garbage

In [151]:
# def productStringCol(frame):
#     frame['product'] = frame['product'].astype(str)
#     return frame


# df_cost = productStringCol(df_cost)
# df_vol = productStringCol(df_volume)

In [157]:
# isYear = df_cost['fy']==2010
# # isIdRaw = df_cost['product_id']==3
# isProd = df_cost['product'].str.contains("Single")

# df_cost[isYear & isProd]